In [1]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp312-cp312-macosx_11_0_arm64.whl size=44092 sha256=6e1290fc9eddd6e5a1c607cb72ea3ce6298415323e36c980731e00d0f6f894b2
  Stored in directory: /Users/shuai/Library/Caches/pip/wheels/77/56/3d/ecbc52138290af7ae8703dabe02cede143ab9cea349cf1aef3
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2
Cloning into 'detectron2'...
remote: Enumerating objects: 15832, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 15832 (delta 24), reused 16 (delta 14), pack-reused 15786 (from 3)
Receiving objects: 100% (15832/15832), 6.40 MiB | 10.19 MiB/s, done.
Resolving deltas: 100% (11529/11529), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
  Using cached cloudpickle-3.1.0-py3-none-any.whl.

In [1]:
import sys, os, distutils.core
sys.path.insert(0, os.path.abspath('./detectron2'))

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

zsh:1: command not found: nvcc
torch:  2.5 ; cuda:  2.5.1
detectron2: 0.6


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {"mask_format": "bitmask"}, "/Users/shuai/Desktop/Mining/Dataset/Labling Job/train_dataset_coco/annotations.json", "/Users/shuai/Desktop/Mining/Dataset/Labling Job/train_dataset_coco")
register_coco_instances("my_dataset_val2", {"mask_format": "bitmask"}, "/Users/shuai/Desktop/Mining/Dataset/Labling Job/val_dataset_coco/annotations.json", "/Users/shuai/Desktop/Mining/Dataset/Labling Job/val_dataset_coco")

In [5]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.MODEL.DEVICE = "cpu"
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

cfg.INPUT.MASK_FORMAT = "bitmask"

In [4]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[12/19 23:07:02 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[12/19 23:07:02 d2.engine.train_loop]: Starting training from iteration 0


/Users/shuai/Desktop/Mining/dev/detectron2/detectron2/data/detection_utils.py:449: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:212.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/Users/shuai/Desktop/Mining/dev/detectron2/detectron2/data/detection_utils.py:449: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be

[12/19 23:08:11 d2.utils.events]:  eta: 0:15:40  iter: 19  total_loss: 3.1  loss_cls: 0.6834  loss_box_reg: 0.8815  loss_mask: 0.6915  loss_rpn_cls: 0.5657  loss_rpn_loc: 0.27    time: 3.3348  last_time: 3.4799  data_time: 0.0767  last_data_time: 0.0011   lr: 1.6068e-05  
[12/19 23:09:17 d2.utils.events]:  eta: 0:14:35  iter: 39  total_loss: 2.581  loss_cls: 0.6196  loss_box_reg: 0.8774  loss_mask: 0.6828  loss_rpn_cls: 0.1401  loss_rpn_loc: 0.2213    time: 3.3079  last_time: 3.1015  data_time: 0.0014  last_data_time: 0.0013   lr: 3.2718e-05  
[12/19 23:10:21 d2.utils.events]:  eta: 0:13:28  iter: 59  total_loss: 2.282  loss_cls: 0.5256  loss_box_reg: 0.8075  loss_mask: 0.6626  loss_rpn_cls: 0.0629  loss_rpn_loc: 0.263    time: 3.2822  last_time: 3.4071  data_time: 0.0013  last_data_time: 0.0017   lr: 4.9367e-05  
[12/19 23:11:27 d2.utils.events]:  eta: 0:12:21  iter: 79  total_loss: 2.237  loss_cls: 0.4781  loss_box_reg: 0.8309  loss_mask: 0.6293  loss_rpn_cls: 0.05558  loss_rpn_loc: 

In [2]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 90429), started 0:00:33 ago. (Use '!kill 90429' to kill it.)

In [6]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[12/19 23:45:46 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


/Users/shuai/miniconda3/envs/blast/lib/python3.12/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_locat

In [35]:
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode
from detectron2.data import DatasetCatalog

# Load the dataset as dataset_dicts
dataset_dicts = DatasetCatalog.get("my_dataset_val2")
my_dataset_metadata = MetadataCatalog.get("my_dataset_val2")

# Load the homography matrix from npy
H = np.load("homography_matrix.npy")

for d in random.sample(dataset_dicts, 2):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=my_dataset_metadata, 
                   scale=1, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    output_image = np.array(out.get_image()[:, :, ::-1], dtype=np.uint8)

    # Retrieve bounding boxes and classes
    pred_boxes = outputs["instances"].pred_boxes
    pred_classes = outputs["instances"].pred_classes

    print("pred_boxes:", pred_boxes)

    for box in pred_boxes.to("cpu"):
        x_min, y_min, x_max, y_max = box.numpy()

        # Bottom-center of the bounding box in the original image
        center_x = (x_min + x_max) / 2
        center_y = y_max

        # Radius of the circle (half the width of the bounding box)
        radius = int((x_max - x_min) / 2)

        # Set the major axis equal to the bounding box width
        major_axis_length = x_max - x_min

        # Define a vertical point for the minor axis calculation
        vertical_point = np.array([center_x, center_y - radius, 1])  # Point directly upward

        # Apply homography to the center and the vertical axis point
        warped_center = H @ np.array([center_x, center_y, 1])
        warped_vertical = H @ vertical_point

        # Normalize homogeneous coordinates
        warped_center /= warped_center[2]
        warped_vertical /= warped_vertical[2]

        # Calculate the minor axis length (distance from center to warped vertical point)
        minor_axis_length = 2 * np.linalg.norm(warped_vertical[:2] - warped_center[:2])  # Double for full axis

        # Draw the ellipse with the original position and adjusted axes
        cv2.ellipse(
            output_image,
            (int(center_x), int(center_y)),  # Original center
            (int(major_axis_length / 2), int(minor_axis_length / 2)),  # Half-lengths of axes
            0,  # No rotation needed
            0, 360,  # Full ellipse
            (0, 0, 255),  # Red color
            thickness=2
        )


    design_map = cv2.imread("drill_hole_map.png", cv2.IMREAD_UNCHANGED)

    output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2BGRA)

    # Warp the design map to align with the drone footage
    aligned_map = cv2.warpPerspective(design_map, H, (output_image.shape[1], output_image.shape[0]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

    # Blend the aligned design map onto the drone image
    # Normalize alpha channel of the aligned map
    alpha_map = aligned_map[:, :, 3] / 255.0  # Normalized alpha values (0.0 to 1.0)

    # Blend each channel (R, G, B)
    for c in range(3):  # Iterate over the R, G, B channels
        output_image[:, :, c] = (
            alpha_map * aligned_map[:, :, c] + (1 - alpha_map) * output_image[:, :, c]
        ).astype(np.uint8)

    # Combine the alpha channels: keep the maximum alpha for each pixel
    output_image[:, :, 3] = np.maximum(output_image[:, :, 3], aligned_map[:, :, 3])
        

    output_filename = "output_" + os.path.basename(d["file_name"])
    success = cv2.imwrite(output_filename, output_image)
    if not success:
        print("Failed to save the image.")
    # cv2.imshow("Predictions", out.get_image()[:, :, ::-1])
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # # Display the image
    # im = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
    # plt.imshow(im)
    # plt.axis('off')  # Turn off axes for better display
    # plt.show()

[12/20 01:08:27 d2.data.datasets.coco]: Loaded 2 images in COCO format from /Users/shuai/Desktop/Mining/Dataset/Labling Job/val_dataset_coco/annotations.json
pred_boxes: Boxes(tensor([[ 852.2872,  324.3633,  931.4574,  480.2841],
        [ 820.7631,  425.8528,  875.2355,  488.3456],
        [ 699.7626,  404.8133,  748.9818,  471.7026],
        [ 416.7763,  444.4831,  445.9590,  488.6767],
        [ 517.1619,  413.8363,  550.4404,  468.5389],
        [ 924.9492,  403.3569,  961.2749,  464.1497],
        [1178.8324,  342.2535, 1274.7996,  487.0500]]))
pred_boxes: Boxes(tensor([[ 777.1945,  194.1928,  902.4446,  487.1743],
        [ 946.6554,  401.0926,  998.2261,  470.3513],
        [1066.6348,  446.6577, 1146.5656,  595.9115],
        [ 470.1970,  320.8749,  528.8103,  470.1872],
        [ 363.2117,  357.7937,  459.0234,  522.9763],
        [ 821.9166,  503.9017,  873.4849,  562.8269],
        [ 903.7746,  346.5542,  945.5742,  461.3372],
        [ 542.6726,  443.5585,  574.6830,  482.6